
# OZON RecSys Baseline - Рекомендательная система для категории Apparel
Этот ноутбук содержит базовое решение для задачи предсказания следующей покупки пользователя в категории одежды, обуви и аксессуаров.

## Задача
- Предсказать топ-100 товаров для каждого пользователя из тестовой выборки
- Метрика оценки: NDCG@100
- Данные: ~38GB в формате parquet, 1.6B взаимодействий, 19M заказов


In [1]:
import pandas as pd
import numpy as np
import glob
from pathlib import Path
from collections import defaultdict
import subprocess
import json
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import dask.dataframe as dd
import gc
import dask

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 1. Загрузка и анализ данных


In [2]:
def load_train_data():
    print("Загружаем тренировочные данные через Dask...")

    orders_path = '/home/root6/python/e_cup/rec_system/data/raw/ml_ozon_recsys_train/final_apparel_orders_data/*/*.parquet'
    tracker_path = '/home/root6/python/e_cup/rec_system/data/raw/ml_ozon_recsys_train/final_apparel_tracker_data/*/*.parquet'
    items_path = '/home/root6/python/e_cup/rec_system/data/raw/ml_ozon_recsys_train/final_apparel_items_data/*.parquet'
    categories_path = '/home/root6/python/e_cup/rec_system/data/raw/ml_ozon_recsys_train_final_categories_tree/*.parquet'

    # --- Загружаем заказы ---
    orders_ddf = dd.read_parquet(orders_path)
    print(f"Найдено файлов заказов: {orders_ddf.npartitions} частей")

    # --- Загружаем взаимодействия ---
    tracker_ddf = dd.read_parquet(tracker_path)
    print(f"Найдено файлов взаимодействий: {tracker_ddf.npartitions} частей")

    # --- Загружаем товары ---
    items_ddf = dd.read_parquet(items_path)
    print(f"Найдено файлов товаров: {items_ddf.npartitions} частей")

    # --- Загружаем категории ---
    categories_ddf = dd.read_parquet(categories_path)
    print(f"Категорий после фильтрации: {categories_ddf.shape[0].compute():,}")

    # --- Возвращаем Dask DataFrame ---
    return orders_ddf, tracker_ddf, items_ddf, categories_ddf

In [3]:
orders_df, tracker_df, items_df, category_df = load_train_data()

Загружаем тренировочные данные через Dask...
Найдено файлов заказов: 28 частей
Найдено файлов взаимодействий: 298 частей
Найдено файлов товаров: 100 частей
Категорий после фильтрации: 7,012


In [4]:
display(orders_df.head(2))
display(orders_df["last_status"].unique().compute())

,item_id,user_id,created_timestamp,last_status,last_status_timestamp,created_date
0,7510328,981,2025-07-02 08:27:17.686,delivered_orders,2025-07-06 16:14:12,2025-07-02
1,136809690,4670,2025-07-02 17:17:30.423,canceled_orders,2025-07-06 19:35:07,2025-07-02


0    proccesed_orders
0     canceled_orders
0    delivered_orders
Name: last_status, dtype: string

In [5]:
orders_df = orders_df[orders_df["last_status"] != "canceled_orders"]
display(orders_df["last_status"].unique().compute())

0    proccesed_orders
0    delivered_orders
Name: last_status, dtype: string

In [6]:
display(tracker_df.head(2))
display(tracker_df["action_type"].unique().compute())

,item_id,user_id,timestamp,action_type,action_widget,date
0,678602,4211601,2025-05-05 18:11:15,page_view,cart.cartSplit,2025-05-05
1,678602,4211601,2025-05-05 18:12:57,view_description,pdp.characteristics,2025-05-05


0              remove
0    view_description
0           page_view
0            favorite
0          unfavorite
0         review_view
0             to_cart
Name: action_type, dtype: string

In [7]:
tracker_df = tracker_df[
    tracker_df["action_type"].isin(["favorite", "to_cart", "page_view", "view_description"])
].drop(columns=["action_widget"], errors="ignore")
display(tracker_df["action_type"].unique().compute())
display(tracker_df.head(2))

0    view_description
0           page_view
0            favorite
0             to_cart
Name: action_type, dtype: string

,item_id,user_id,timestamp,action_type,date
0,678602,4211601,2025-05-05 18:11:15,page_view,2025-05-05
1,678602,4211601,2025-05-05 18:12:57,view_description,2025-05-05


In [8]:
items_df = items_df.drop(columns=["variant_id", "model_id"], errors="ignore")
display(items_df.head(2))

,item_id,itemname,attributes,fclip_embed,catalogid
0,400301,Очки солнцезащитные,"[{'attribute_name': 'Type', 'attribute_value':...","[0.32023853, 0.11766258, -0.784552, 0.4168199,...",17080
1,678602,Футболка GOLDUSTIM Футболка женская,"[{'attribute_name': 'Type', 'attribute_value':...","[0.4360095, 0.6339668, 0.029741853, 0.09847632...",7508


In [9]:
display(orders_df.head(2))
display(orders_df["last_status"].unique().compute())

,item_id,user_id,created_timestamp,last_status,last_status_timestamp,created_date
0,7510328,981,2025-07-02 08:27:17.686,delivered_orders,2025-07-06 16:14:12,2025-07-02
3,42322137,12860,2025-07-02 00:40:52.503,delivered_orders,2025-07-04 19:44:48,2025-07-02


0    proccesed_orders
0    delivered_orders
Name: last_status, dtype: string

In [10]:
category_df.head(5)

,catalogid,catalogpath,ids
0,15500,"[{'id': 15500, 'name': 'Электроника', 'fullNam...","[15500, -1]"
1,15501,"[{'id': 15501, 'name': 'Телефоны и смарт-часы'...","[15501, 15500, -1]"
2,15502,"[{'id': 15502, 'name': 'Смартфоны', 'fullName'...","[15502, 15501, 15500, -1]"
3,15511,"[{'id': 15511, 'name': 'Аксессуары для смартфо...","[15511, 15501, 15500, -1]"
4,15892,"[{'id': 15892, 'name': 'Чехлы', 'fullName': 'Ч...","[15892, 15511, 15501, 15500, -1]"


In [11]:
# items_df
items_df_apparel = items_df[items_df['catalogid'].isin(category_df['catalogid'])]
items_df_apparel.head(5)

,item_id,itemname,attributes,fclip_embed,catalogid
0,400301,Очки солнцезащитные,"[{'attribute_name': 'Type', 'attribute_value':...","[0.32023853, 0.11766258, -0.784552, 0.4168199,...",17080
1,678602,Футболка GOLDUSTIM Футболка женская,"[{'attribute_name': 'Type', 'attribute_value':...","[0.4360095, 0.6339668, 0.029741853, 0.09847632...",7508
2,746401,Худи Patagonia,"[{'attribute_name': 'Type', 'attribute_value':...","[0.47452784, -0.4727797, -0.51971936, -0.08221...",7555
3,872829,"Комплект носков adidas 3S C Spw Mid 3P, 3 пары","[{'attribute_name': 'Type', 'attribute_value':...","[-0.06987186, -2.2802832, -0.5829071, -0.33674...",36563
4,893255,Жилет Buy & Style жилеты,"[{'attribute_name': 'Type', 'attribute_value':...","[0.12488763, 0.5466293, 0.29031003, 0.21499962...",7535


## 2. Exploratory Data Analysis (EDA)


In [12]:
print("АНАЛИЗ ЗАКАЗОВ")
print("=" * 50)

# Общее количество строк
total_orders = orders_df.shape[0].compute()
print(f"Общее количество заказов: {total_orders:,}")

# Уникальные пользователи и товары
print(f"Уникальных пользователей: {orders_df['user_id'].nunique().compute():,}")
print(f"Уникальных товаров: {orders_df['item_id'].nunique().compute():,}")

# Если есть колонка created_date
if 'created_date' in orders_df.columns:
    orders_df['created_date'] = orders_df['created_date'].map_partitions(pd.to_datetime)
    min_date, max_date = dask.compute(
        orders_df['created_date'].min(),
        orders_df['created_date'].max()
    )
    print(f"Период данных: {min_date} - {max_date}")
    del min_date, max_date

print("\nРаспределение статусов заказов:")
status_counts = orders_df.groupby("last_status").size().compute()
for status, count in status_counts.items():
    percentage = count / total_orders * 100
    print(f"  {status}: {count:,} ({percentage:.1f}%)")

# после анализа - чистим
del status_counts, total_orders
gc.collect()

АНАЛИЗ ЗАКАЗОВ
Общее количество заказов: 1,045,162
Уникальных пользователей: 327,596
Уникальных товаров: 542,602
Период данных: 2025-07-02 00:00:00 - 2025-07-15 00:00:00

Распределение статусов заказов:
  delivered_orders: 484,645 (46.4%)
  proccesed_orders: 560,517 (53.6%)


41

In [13]:
print("\nАНАЛИЗ ВЗАИМОДЕЙСТВИЙ")
print("=" * 50)

# Общее количество строк
total_tracker = tracker_df.shape[0].compute()
print(f"Общее количество взаимодействий: {total_tracker:,}")

# Уникальные пользователи и товары
print(f"Уникальных пользователей: {tracker_df['user_id'].nunique().compute():,}")
print(f"Уникальных товаров: {tracker_df['item_id'].nunique().compute():,}")

print("\nРаспределение типов действий:")
action_counts = tracker_df.groupby("action_type").size().compute()

for action, count in action_counts.items():
    percentage = count / total_tracker * 100
    print(f"  {action}: {count:,} ({percentage:.1f}%)")

# после анализа - чистим
del action_counts, total_tracker
gc.collect()


АНАЛИЗ ВЗАИМОДЕЙСТВИЙ
Общее количество взаимодействий: 1,617,426,792
Уникальных пользователей: 1,010,758
Уникальных товаров: 4,931,453

Распределение типов действий:
  favorite: 55,403,595 (3.4%)
  page_view: 1,257,211,877 (77.7%)
  view_description: 239,133,345 (14.8%)
  to_cart: 65,677,975 (4.1%)


41

In [14]:
print("\nАНАЛИЗ ТОВАРОВ")
print("=" * 50)

# Общее количество товаров
total_items = items_df_apparel.shape[0].compute()
print(f"Общее количество товаров: {total_items:,}")

# С эмбеддингами
items_with_embed = (~items_df_apparel['fclip_embed'].isna()).sum().compute()
print(f"Товары с эмбеддингами: {items_with_embed:,}")

# С атрибутами
if 'attributes' in items_df_apparel.columns:
    items_with_attr = (~items_df_apparel['attributes'].isna()).sum().compute()
    print(f"Товары с атрибутами: {items_with_attr:,}")

print("\nТоп-10 категорий:")
cat_counts = items_df_apparel['catalogid'].value_counts().nlargest(10).compute()

for cat_id, count in cat_counts.items():
    print(f"  Категория {cat_id}: {count:,}")

# после анализа - чистим
del total_items, items_with_embed, cat_counts
if 'items_with_attr' in locals():
    del items_with_attr
gc.collect()


АНАЛИЗ ТОВАРОВ
Общее количество товаров: 6,439,422
Товары с эмбеддингами: 6,439,422
Товары с атрибутами: 6,439,422

Топ-10 категорий:
  Категория 7559: 413,637
  Категория 7508: 267,444
  Категория 36484: 225,336
  Категория 7660: 219,858
  Категория 7502: 199,919
  Категория 7512: 120,846
  Категория 7511: 108,535
  Категория 7530: 106,773
  Категория 17002: 105,158
  Категория 7644: 101,622


41

## 3. Загрузка тестовых пользователей

In [15]:
def load_test_users():

    print("Загружаем тестовых пользователей...")
    
    test_files = glob.glob('/home/root6/python/e_cup/rec_system/data/raw/ml_ozon_recsys_test_for_participants/test_for_participants/*.parquet')
    all_users = set()
    
    for file_path in tqdm(test_files, desc="Обработка тестовых файлов"):
        df = pd.read_parquet(file_path)
        all_users.update(df['user_id'].unique())
    
    print(f"Найдено уникальных тестовых пользователей: {len(all_users):,}")
    return list(all_users)

test_users = load_test_users()

Загружаем тестовых пользователей...


Обработка тестовых файлов: 100%|██████████| 1/1 [00:00<00:00, 13.34it/s]

Найдено уникальных тестовых пользователей: 470,347


In [16]:
test_users[:5]

[np.int32(1),
 np.int32(3145730),
 np.int32(3145731),
 np.int32(1048580),
 np.int32(4194310)]

## 4. Построение модели